<a href="https://colab.research.google.com/github/saribasmetehan/Create-custom-ner-model-with-Spacy/blob/main/custom_ner4model_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_trf/resolve/main/tr_core_news_trf-any-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.2/849.2 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 35.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Usin

In [ ]:
import requests
import json

url = "https://raw.githubusercontent.com/turkish-nlp-suite/Atis_Turkish/main/json/atis_train.json"

response = requests.get(url)

with open("atis_train.json", "wb") as f:
    f.write(response.content)

with open("atis_train.json") as f:
    data = json.load(f)

TRAIN_DATA = []
for item in data:
    text = item["text"]
    entities = [(entity["start"], entity["end"], entity["entity"]) for entity in item["entities"]]
    TRAIN_DATA.append((text, {"entities": entities}))

In [ ]:

url = "https://raw.githubusercontent.com/turkish-nlp-suite/Atis_Turkish/main/json/atis_dev.json"

response1 = requests.get(url)

with open("atis_dev.json", "wb") as f:
    f.write(response1.content)

with open("atis_dev.json") as f:
    data = json.load(f)

DEV_DATA = []
for item in data:
    text = item["text"]
    entities = [(entity["start"], entity["end"], entity["entity"]) for entity in item["entities"]]
    DEV_DATA.append((text, {"entities": entities}))

In [ ]:
DEV_DATA

[("Pittsburgh'tan Atlanta'ya 25 Nisan'da kalkan ve 6 Mayıs'ta dönen en ucuz uçuşu istiyorum",
  {'entities': [(0, 14, 'fromloc.city_name'),
    (15, 25, 'toloc.city_name'),
    (26, 28, 'depart_date.day_number'),
    (29, 37, 'depart_date.month_name'),
    (48, 49, 'return_end_time.time'),
    (50, 58, 'return_date.month_name'),
    (65, 72, 'cost_relative')]}),
 ("Memphis'ten Seattle'ye en geç akşam saat 3'te varacak bir uçuş istiyorum",
  {'entities': [(0, 11, 'fromloc.city_name'),
    (12, 22, 'toloc.city_name'),
    (30, 35, 'arrive_time.period_of_day'),
    (41, 45, 'arrive_time.time')]}),
 ("Bana gelecek salı kalkan ve ertesi gün dönen Chicago'dan Detroit'e gidiş dönüş uçuşlarını gösterin",
  {'entities': [(5, 12, 'depart_date.date_relative'),
    (13, 17, 'depart_date.day_name'),
    (28, 38, 'return_date.today_relative'),
    (45, 56, 'fromloc.city_name'),
    (57, 66, 'toloc.city_name')]}),
 ('yemek için "d s" ne anlama geliyor', {'entities': [(0, 5, 'meal')]}),
 ("Bana Atlant

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("tr") # load a new spacy model TURKISH
nlp = spacy.load("tr_core_news_trf") # load other spacy model TURKISH

db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir(r'/content/drive/MyDrive/custom_ner4')
db.to_disk("/content/drive/MyDrive/custom_ner4/train.spacy") # save the docbin object

100%|██████████| 4274/4274 [00:00<00:00, 7009.22it/s]


In [ ]:
for text, annot in tqdm(DEV_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir(r'/content/drive/MyDrive/custom_ner4')
db.to_disk("/content/drive/MyDrive/custom_ner4/dev.spacy") # save the docbin object

100%|██████████| 572/572 [00:00<00:00, 4398.35it/s]


In [ ]:
%cd /content/drive/MyDrive/custom_ner4
kbbbbbbbbb
!python -m spacy init fill-config base_config.cfg config.cfg

/content
2024-05-03 15:14:47.111821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 15:14:47.111873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 15:14:47.113203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 15:14:48.131346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train /content/drive/MyDrive/custom_ner4/config.cfg --output ./output --paths.train /content/drive/MyDrive/custom_ner4/train.spacy --paths.dev /content/drive/MyDrive/custom_ner4/dev.spacy --gpu-id 0

2024-05-03 15:20:37.182808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 15:20:37.182858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 15:20:37.184246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 15:20:38.239393: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2024-05-03 15:20:41,488] [INFO] Set up nlp object from config
[2024-05-03 15:20:41,517] [INFO] Pipeline: ['tr

In [ ]:
nlp1 = spacy.load("/content/drive/MyDrive/custom_ner4/output/model-best") #load the best model
doc = nlp1("Nashville'den Seattle'ye giden saat 3'ten önce varışlı bir uçuş istiyorum") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter